In [36]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [37]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])
target.shape

(506, 1)

In [38]:
property_stats = np.ndarray(shape=(1, 11))
property_stats = features.mean().values.reshape(1, 11)
features.mean()

CRIM         3.613524
ZN          11.363636
CHAS         0.069170
NOX          0.554695
RM           6.284634
DIS          3.795043
RAD          9.549407
TAX        408.237154
PTRATIO     18.455534
B          356.674032
LSTAT       12.653063
dtype: float64

In [39]:
regr =LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [43]:
def get_log_astimate(nr_rooms,
                    students_per_classroom,
                    next_to_river=False,
                    high_confidence=True):
    # Configure property
    property_stats[0][4] = nr_rooms
    property_stats[0][8] = students_per_classroom
    
    if next_to_river:
        property_stats[0][2] = 1
    else:
        property_stats[0][2] = 0
    
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [45]:
get_log_astimate(3, 20, next_to_river=True, high_confidence=False)

(2.7767581914803965, 2.9642703266775268, 2.589246056283266, 68)

In [52]:
ZILLOW_MEDIAN_PRICE = 583.2 # 659.8 on 2021/03/08
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_astimate(9, students_per_classroom=15,
                                              next_to_river=False, high_confidence=False)

# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is:')
print(f'USD {rounded_low} at the lower and to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0.
At 68% confidence the valuation range is
USD 685000.0 at the lower and to USD 997000.0 at the high end.


In [63]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom
    chas -- True if the property is next to the river
    large_range -- True for a 95% prediction interval, False for a 68% interval    
    """
    
    if rm < 1 or ptratio < 0:
        print('Unrealistic')
        return
    
    log_est, upper, lower, conf = get_log_astimate(rm, ptratio,
                                                  next_to_river=chas, high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is:')
    print(f'USD {rounded_low} at the lower and to USD {rounded_hi} at the high end.')

In [62]:
get_dollar_estimate(rm=0, ptratio=-60, chas=True)

Unrealistic


In [64]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property value is 885000.0.
At 95% confidence the valuation range is:
USD 609000.0 at the lower and to USD 1288000.0 at the high end.
